$
\newcommand{\reg}{{}^\textsuperscript{\textcircled{\textsc r}}}
\newcommand{\ml}{{\bf L}}
$
# Solving Nonlinear Equations with Iterative Methods: <br> Solvers and Examples in Julia<br>   C. T. Kelley

Copyright C. T. Kelley, 2020

To Betty Thomas Kelley

## Contents

[Preface and Software](#Preface_and_Software)

- [Preface for the notebook](#Preface-for-the-notebook)

- [Preface for the print book](#Preface-for-the-print-book)

- [Learning Julia](#Learning-Julia)

- [How to get the software](#How-to-get-the-software)

- [Software License](LICENSE-CODE.md)

- [Text License (ie the notebook)](LICENSE-TEXT.md)

[Chapter 1: Introduction](SIAMFANLCh1.ipynb)

- [Solvers for Chapter 1: __nsolsc.jl__ and __ptcsolsc.jl__](SIAMFANLCh1s.ipynb)

[Chapter 2: Finding the Newton Step with Gaussian Elimination](SIAMFANLCh2.ipynb)

- [Solvers for Chapter 2: nsold.jl and ptcsol.jl](SIAMFANLCh2s.ipynb)

# Preface and Software

## Preface for the notebook

This notebook is a more interactive version of the print book. I encourage you to play with the examples, 
make up your own, and explore the algorithmic parameters. I hope it's more fun to play with this than to just
read text. However, the print book is more useful when you need to look something up or want to bookmark a few
pages while working on your own projects. 

The older book 
<cite data-cite="ctk:newton"><a href="siamfa.html#ctk:newton">(Kel03)</cite>
had listings of source code in an appendix and descriptions of the solvers in the main part of the text. 
We do that differently here. The description of the algorithms in the notebook is taken from the print book, but for the discussion of the software and most of the examples, we convert the notebook to $\LaTeX$ and incorporate that directly into the chapters of the print book.

I hope that the three parts of this project, print book, notebook, and the solver package make one coherent thing.    


## Preface for the print book


This book on solvers for nonlinear equations
is a user-oriented guide to algorithms and implementation.
It is a sequel to 
<cite data-cite="ctk:newton"><a href="siamfa.html#ctk:newton">(Kel03)</cite>,
which used MATLAB® for 
the solvers and examples. This book uses Julia 
<cite data-cite="Juliasirev"><a href="siamfa.html#Juliasirev">(BEKS17)</cite>
and adds new material
on pseudo-transient continuation, mixed-precision solvers, and
Anderson acceleration.

Its purpose is to show, via algorithms
in pseudocode and Julia and with several examples, how one
can choose an appropriate iterative method for a given problem and
write an efficient solver or apply one written by others.

This book is intended to
complement my larger book complement my larger book 
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
which focuses on in-depth
treatment of convergence theory, but does not discuss the details of
solving particular problems, implementation in any particular language,
or evaluating a solver for a given problem.

The computational examples in this book were
done with Julia 1.4 on various Apple computers.
The Julia codes for the solvers are in a Julia package.
I provide a Jupyter notebook to enable the reader to run all
the examples in the book and play with them.

I have found that Julia is an excellent environment for a project
like this, which unifies a print book, a package of solvers, and an
interactive notebook. I have used all the solvers in my own research
and many of the examples come from that research.

I have written the codes with a goal of
readability by a Julia novice who knows some
numerical analysis.
I have also made the algorithmic parameters very easy to find.
I have sacrificed a
good deal of abstraction, some generality, and a bit of performance
for that clarity.
The solvers
were designed for production work on desktop and laptop computers
to solve small-to medium-scale
problems having at most a a few tens of thousands of unknowns.
Very large-scale problems on leadership-class computers
are best done with software
(Trilinos <cite data-cite="trilinos"><a href="siamfa.html#trilinos">(HBH<sup>+</sup>05)</cite>,
PETSc <cite data-cite="petsc-web-page"><a href="siamfa.html#petsc-web-page">(BAA<sup>+</sup>19)</cite>
SunDials <cite data-cite="sundials"><a href="siamfa.html#sundials">(HBG<sup>+</sup>05)</cite>)
designed for that purpose.


We assume that the reader has a good understanding of elementary
numerical analysis at the level of 
<cite data-cite="atkelem"><a href="siamfa.html#atkelem">(Atk89)</cite>,
and of numerical linear algebra at the level of 
<cite data-cite="demmel"><a href="siamfa.html#demmel">(Dem97)</cite>,
<cite data-cite="IIbook"><a href="siamfa.html#IIbook">(Ips09)</cite>, and
<cite data-cite="trefbau"><a href="siamfa.html#trefbau">(TB96)</cite>. A student who
has had, or is taking, a course from one of these books is well prepared.
We will also use some deeper results from
numerical linear algebra and will refer to 
<cite data-cite="higham"><a href="siamfa.html#higham">(Hig96)</cite>
for those.

The examples are closely coupled to the text and
the reader will get the most out of
this book if she/he has an elementary knowledge of Julia.
If the reader has good skills in numerical work in another high-level
language then learning Julia at the level one needs to work
through this
book is not difficult and can be done while reading the book.

The reader should know (or learn!) Julia and its ecosystem
well enough to use the package manager,
install packages, use modules, start a notebook, and to do basic tasks in
numerical methods (LU, SVD, QR) in Julia. You should also know
how to use github to clone repositories and put
them where they need to go. A reader with that minimal knowledge of
Juila should be able to understand the codes,
play with the algorithms, and wreak havoc.


Unfortunately, there is no introduction to the language aimed at
the numerical analyst, as there is for Matlab 
<cite data-cite="nick_matlab"><a href="siamfa.html#nick_matlab">(HH05)</cite>.
One very good reference is the paper 
<cite data-cite="Juliasirev"><a href="siamfa.html#Juliasirev">(BEKS17)</cite>.
The full manual 
<cite data-cite="Julia20"><a href="siamfa.html#Julia20">(Pro20)</cite> and the online guide
<cite data-cite="JuliaWeb"><a href="siamfa.html#JuliaWeb">(Ame20)</cite>
also very useful.
    

Parts of this book are based on research supported by the National
Science Foundation,
and the Army Research Office, most recently
by NSF grants
OAC-1740309,
DMS-1745654,
DMS-1906446,
and
ARO grant W911NF-16-1-0504.

Any opinions, findings, and conclusions or
recommendations expressed in this material are those of the author and
do not necessarily reflect the views of the National
Science Foundation
or the Army Research Office.

In addition to the many people who influenced
<cite data-cite="ctk:newton"><a href="siamfa.html#ctk:newton">(Kel03)</cite> and
<cite data-cite="ctk:roots"><a href="siamfa.html#ctk:roots">(Kel95)</cite>,
I want
to particularly thank
Jerry Bernholc,
Wei Bian,
Emil Briggs,
Xiaojun Chen,
Kevin Clarno,
Austin Ellis,
Tom Evans,
Elizabeth Greenspan,
Steven Hamilton,
Nick Higham,
Ilse Ipsen,
Elena Jakubikova,
Randy LeVeque,
Lin Lin,
Wenchang Lu,
Juan Meza,
Zack Morrow,
James Nance,
Chung-Wei Ng,
Sheehan Olver,
Roger Pawlowski,
Liqun Qi,
Stuart Slattery,
Alex Toth,
Homer Walker,
Jeff Willert,
Chao Yang,
and a few hundred MA 784 students
for their inspiration as I got this project into shape.
    
C. T. Kelley<br>
Raleigh, North Carolina<br>
May 2020


## Learning Julia

Unfortunately, there is no introduction to the language aimed at
the numerical analyst, as there is for Matlab 
<cite data-cite="nick_matlab"><a href="siamfa.html#nick_matlab">(HH05)</cite>.
There are many online resources for Julia and you should take some care to
stick to the ones for version 1.0 and above.
    
    
If you are a Julia novice, it will take some experimentation to get
comfortable. One very good reference is the paper 
<cite data-cite="Juliasirev"><a href="siamfa.html#Juliasirev">(BEKS17)</cite>. You should read this first.
The full manual 
<cite data-cite="Julia20"><a href="siamfa.html#Julia20">(Pro20)</cite> and the online guide
<cite data-cite="JuliaWeb"><a href="siamfa.html#JuliaWeb">(Ame20)</cite>
also very useful. If you are coming from Matlab, your best bet is to start off by
learning
how linear algebra works in Julia. The Wiki 
<cite data-cite="JuliaWiki"><a href="siamfa.html#JuliaWiki">(Con20)</cite>
is good.
    
    
There are many resources at
<https://julialang.org/learning>.

- There are general introductions to
Julia that are aimed a more general audiences or introductory computer
science courses. A good example of such a book is 
<cite data-cite="LauensDowney"><a href="siamfa.html#LauensDowney">(LD19)</cite>,
which is also available at
<https://benlauwens.github.io/ThinkJulia.jl/latest/book.html>.


- The Julia youtube channel <https://www.youtube.com/channel/UC9IuUwwE2xdjQUT_LMLONoA> has several useful tutorials.


- Two useful and fast introcutions are the video
<https://www.youtube.com/watch?v=8h8rQyEpiZA&t> and the collection of web pages
<cite data-cite="JuliaWeb"><a href="siamfa.html#JuliaWeb">(Ame20)</cite> <https://techytok.com/from-zero-to-julia>.


- The discourse at __julialang.org__ <https://discourse.julialang.org> is a welcoming environment for the novice.

    

## How to get the software

If you are reading this, you've found the notebook and probably have installed the package. __If you have not installed the package, type this at the REPL prompt or in a code cell in the notebook.__

import Pkg; Pkg.add(url="https://github.com/ctkelley/SIAMFANLEquations.jl")


The notebooks for the individual chapters get the packages organized in the first code window, which you __must run!__ Your best bet is to do __run all__ as the first step before playing with the notebooks. Nothing in the notebooks should take a long time to run.

I am assuming you've followed the directions in the README for the repo and cloned the whole works. I will use the module /src/NotebookSIAMFANL.jl in the notebooks. That module has the examples from the book organized in a sensible way.

The project has three parts:

- this notebook
- the Julia package SIAMFANLEquations.jl
- the test problems

The first cell in each notebook is an invisible markdown cell with the LaTeX commands I need. Do not mess with that.

As you will see, that first code cell in each chapter has the single line

```julia
include("fanote_init.jl")
```
at the start.

This is a Julia script that contains

```julia
using SIAMFANLEquations
using SIAMFANLEquations.TestProblems
using LinearAlgebra
using PyPLot
push!(LOAD_PATH,"./src")
using NotebookSIAMFANL
```
The script gets the dependencies and the functions for the examples where they need to be.

These two cells __must__ run first. That's one reason that __run all__ before starting is a good idea.


